In [1]:
from bitcom.client.market_client import MarketClient
from bitcom.client.account_client import AccountClient
from bitcom.client.order_client import OrderClient
from bitcom.utils import *
from bitcom.constant import * 
import json
import math
import numpy as np
import asyncio
import websockets
import json
import requests
import dateutil.parser as dp
from dateutil import tz
import hmac
import base64
import zlib
import datetime
import pandas as pd
import time
import nest_asyncio
import math
from decimal import Decimal, getcontext, setcontext,ROUND_DOWN, ROUND_UP,ROUND_CEILING
nest_asyncio.apply()

def get_timestamp():
    now = datetime.datetime.now()
    t = now.isoformat("T", "milliseconds")
    return t + "Z"

time_1 = get_timestamp()

def precfloat(num,digi):
    return int(round(float(num)*math.pow(10,digi)))/math.pow(10,digi)

In [2]:
def precfloat(num,digi):
    return int(round(float(num)*math.pow(10,digi)))/math.pow(10,digi)

def isoStr2utc8Str(isoStr):
    utc=datetime.datetime.strptime(isoStr,"%Y-%m-%dT%H:%M:%S.%fZ")
    utc8Time =utc.replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return datetime.datetime.strptime(utc8Time,"%Y-%m-%d %H:%M:%S.%f")

def get_position( use_currency,*args,**kwargs):
    param_map = {'currency': use_currency,'instrument_id': swap_instrument_id }
    if len(account_client.get_user_positions(param_map)['data'])==0:
        Result_Position=0
    else:    
        Result_Position = float(account_client.get_user_positions(param_map)['data'][0]['qty'])  
    #仓位,带方向(期权仓位单位为BTC, 期货仓位单位为USD),正数为多仓，负数为空仓。   
    return Result_Position 

def take_reduce_order(mode_holding_reduce,*args,**kwargs):    #平仓,市价单送掉
    swap_instrument_id=mode_holding_reduce['swap_instrument_id']
    upper_reduce_size=mode_holding_reduce['upper_reduce_size']
    upper_price=mode_holding_reduce['upper_price']
    
    param_map ={'side':'sell', 'order_type':'limit','time_in_force':'gtc','qty':upper_reduce_size,
                'price':upper_price, 'instrument_id':swap_instrument_id}
    order_result = order_client.place_new_order(param_map)    
   
    return order_result
 
def take_add_order(mode_holding_add,*args,**kwargs):   #开仓,市价单送掉
    swap_instrument_id=mode_holding_add['swap_instrument_id']
    lower_add_size=mode_holding_add['lower_add_size']
    lower_price=mode_holding_add['lower_price']
    
    
    param_map ={'side':'buy', 'order_type':'limit','time_in_force':'gtc','qty':lower_add_size,
                'price':lower_price, 'instrument_id':swap_instrument_id}
    order_result = order_client.place_new_order(param_map)
    
    return order_result
        
    
def take_order_iceberg(mode_take_order_iceberg,*args,**kwargs):    #bit.com没有看到冰山单
    swap_instrument_id=mode_take_order_iceberg['swap_instrument_id']
    iceberg_size=mode_take_order_iceberg['iceberg_size']
    spot_price_limit=mode_take_order_iceberg['spot_price_limit']
    iceberg_ave_open_size=mode_take_order_iceberg['iceberg_ave_open_size']
    
    A = spotAPI.take_order_algo(instrument_id=swap_instrument_id, mode='1', order_type='3', size=iceberg_size, side='buy', 
                                algo_variance='0.002',avg_amount=iceberg_ave_open_size, limit_price=spot_price_limit)
           
    return A
    
def sendmessage(message):
    url = 'https://oapi.dingtalk.com/robot/send?access_token=36add27c7eaaedbb35cd83cb42e14f72b31683526e6a22524ab2f82a937c3fd1'
    HEADERS = {"Content-Type": "application/json ;charset=utf-8 "}
    message = message
    String_textMsg = {
        'msgtype':'text',
        'text':{'content':message},
   #     'at':{'atMobiles':{'15201106731'}, 'isAtAll':0 #要＠所有人就是1  }
               }
    String_textMsg = json.dumps(String_textMsg)
    res = requests.post(url, data=String_textMsg, headers=HEADERS)
    
    

In [3]:
if __name__ == '__main__':
    USER1_ACCESS_KEY="" 
    USER1_SECRET_KEY=""
    USER1_HOST='https://api.bit.com' #主网

    #eric给的测试url
    #curl= "https://betaapi.bitexch.dev/v1/funding_rate_history?instrument_id=BTC-PERPETUAL&start_time=1603260000000&end_time=1603346400000&history_type=1H"

    market_client = MarketClient(url=USER1_HOST, access_key=USER1_ACCESS_KEY, secret_key=USER1_SECRET_KEY)
    order_client = OrderClient(url=USER1_HOST, access_key=USER1_ACCESS_KEY, secret_key=USER1_SECRET_KEY)
    account_client = AccountClient(url=USER1_HOST, access_key=USER1_ACCESS_KEY, secret_key=USER1_SECRET_KEY)

    swap_instrument_id='BTC-PERPETUAL'
    use_currency='BTC'
    Tick_digit=2
    notional_size=20


In [ ]:
Total_Money=3000
holding_reduce_mode='off'
holding_add_mode='off'
holding_change_mode='off'
perc_levels=[]

Machine_start_time= datetime.datetime.now()
Fundclock=Machine_start_time
Open_position_clock=Machine_start_time
Open_position_clock_delta=datetime.timedelta(hours=1)
Fundclockdelta=datetime.timedelta(minutes=5)
Position_building_endtime=Machine_start_time+Open_position_clock_delta
Nowtime=datetime.datetime.now()

#找出Tick_size
Tick_size=None
param_map = { 'currency': use_currency, 'category':'future' }
swap_instruments = market_client.get_instruments(param_map)
for i in swap_instruments['data']:
    if i['instrument_id']==swap_instrument_id:
        Tick_size=precfloat(i['price_step'],Tick_digit)

#找出总距离跟24小时中间价
param_map = { 'instrument_id':swap_instrument_id }
ticker_result = market_client.get_tickers(param_map)   
high_24h=precfloat(float(ticker_result['data']['high24h']),Tick_digit)
low_24h=precfloat(float(ticker_result['data']['low24h']),Tick_digit)
total_grids_length=precfloat((high_24h-low_24h),Tick_digit)         #总距离
middle_price_24h=precfloat((high_24h+low_24h)/2,Tick_digit)

#找出 perc_levels
#找格宽,收集24小时的5分钟k线
timefor1De = int(round(time.time()*1000))
dt=str(datetime.datetime.now()-datetime.timedelta(hours=24))
timefor1Ds= int(time.mktime(time.strptime(dt, '%Y-%m-%d %H:%M:%S.%f')))*1000
param_map = { 'instrument_id':swap_instrument_id,'start_time':timefor1Ds , 'end_time':timefor1De, 'timeframe_min':'5' }
kline_result = market_client.get_klines(param_map)
close_price=[]
for i in kline_result['data']['close']:
    close_price.append(precfloat(i,Tick_digit))
grid_width=(np.std(close_price,ddof=1))         #格宽  
step_num=round(total_grids_length/grid_width)    #暂无用到
perc_levels=[precfloat(x,Tick_digit) for x in np.arange(high_24h, low_24h, -1*grid_width)]     #总距离分几格
if (high_24h-grid_width*step_num)/grid_width>=0.7:    #判断最后一格格宽是否太小,可否放入
    perc_levels.append(low_24h)

print('perc_levels',perc_levels)

#现在价格
param_map = { 'instrument_id': swap_instrument_id, 'level': '5'}
swap_depth_result = market_client.get_orderbooks(param_map)
ask1=precfloat(float(swap_depth_result['data']['asks'][0][0]),Tick_digit)
bid1=precfloat(float(swap_depth_result['data']['bids'][0][0]),Tick_digit)
swap_present_price=precfloat((ask1+bid1)/2,Tick_digit)

#找出location_index
location_index=None  #先随便设location_index
#找出在那个节点 ,先设出i
for i in range(len(perc_levels)):
    if swap_present_price > perc_levels[i]:                   # 如果大于某个网格价
        location_index = i        
        break

print('location_index',location_index)
        
#找出在那个中间价的下条线  
middle_price_index=None   #先乱写
for j in range(len(perc_levels)):
    if middle_price_24h > perc_levels[j]:                   # 如果大于某个网格价
        middle_price_index = j
        break
        
        
#计算Target_Amount
Target_Amount=Total_Money
#计算本层（过下线)Target_holding_percent
Target_holding_percent_downline=location_index/(len(perc_levels)-1)    #先找出当下的location_index 然后算target_holding_percent
#计算上层（过上线)Target_holding_percent
Target_holding_percent_upline=(location_index-1)/(len(perc_levels)-1) 
#冰山百分比
iceberg_percent=(middle_price_index-1)/(len(perc_levels)-1)
#找出目前持仓多少
swap_position = get_position(use_currency)
position_holding_rate=swap_position/Target_Amount


#找出上下价格线
if location_index>0:
    if location_index!=len(perc_levels)-1:
        upper_price = perc_levels[location_index-1]
    elif location_index==len(perc_levels)-1:
        upper_price = perc_levels[location_index-2]    #在最后一格的话 卖出线往上挂两格            

if location_index<len(perc_levels):
    lower_price = perc_levels[location_index]    #<<<<<<<<<<<<<<<

print('Target_Amount_1',Target_Amount)
    
#找出上线的出手size
print('position_holding_rate-Target_holding_percent_upline_1',position_holding_rate-Target_holding_percent_upline)
if precfloat(position_holding_rate-Target_holding_percent_upline,5)>0:
    holding_change_mode='on'
    
    upper_reduce_size=round((position_holding_rate-Target_holding_percent_upline)*Target_Amount)
    if upper_reduce_size !=0:
        holding_reduce_mode='on'
    mode_holding_reduce={'swap_instrument_id':swap_instrument_id,     
                          'upper_reduce_size':str(upper_reduce_size),        
                          'upper_price':str(upper_price)}
    
    print('holding_reduce_mode_1',holding_reduce_mode)
    
#找出下线的出手size
print('Target_holding_percent_downline-position_holding_rate_1',Target_holding_percent_downline-position_holding_rate)
if precfloat(Target_holding_percent_downline-position_holding_rate,5)>0:
    holding_change_mode='on'
    lower_add_size=round((Target_holding_percent_downline-position_holding_rate)*Target_Amount)   
    if lower_add_size !=0 and location_index>=middle_price_index:
        holding_add_mode='on'
    mode_holding_add={'swap_instrument_id':swap_instrument_id,     
                      'lower_add_size':str(lower_add_size),        
                      'lower_price':str(lower_price)}  
    
    print('lower_add_size',lower_add_size)
    print('holding_add_mode_1',holding_add_mode)


if holding_change_mode=='on':
    
    #要先撤单,这里是全撤  
    param_map = { 'currency':use_currency, 'category':'future'}
    result_uncomplete_order = order_client.get_open_orders(param_map) 
    order_cancel_list=[]
    if len(result_uncomplete_order ['data']) !=0:
        A=result_uncomplete_order ['data']
        for i in A:
            order_cancel_list.append(i)
        idd=[]
        for i in order_cancel_list:
            idd.append(i['order_id']) 
        if idd != []:
            for q in range(len(idd)):
                param_map = { 'currency':use_currency,'category':'future','order_id': idd[q]}        
                cancel_orders_response=order_client.cancel_orders(param_map)  

    #下单
    if holding_reduce_mode=='on':        
        holding_reduce_order=take_reduce_order(mode_holding_reduce)    #此时下了单,但不是成交了  
        print('holding_reduce_order',holding_reduce_order)
    if holding_add_mode=='on':
        holding_add_order=take_add_order(mode_holding_add)    #此时下了单,但不是成交了 
        print('holding_add_order',holding_add_order)


location_index_change=location_index   #没有用到    
perc_levels_new=perc_levels   #为了格宽的设置改变做准备

#发钉钉
tutu= "bit.com_BTC_PERP网格初始位置在第"+str(location_index)+"格"+str(time_1)
sendmessage(tutu)

tutu= "bit.com_BTC_PERP网格Target_Amount为"+str(Target_Amount)
sendmessage(tutu)

tutu= "bit.com_BTC_PERP网格现在为"+str(perc_levels)
sendmessage(tutu)

tutu= "bit.com_BTC_PERP网格现在持仓％为"+str(position_holding_rate)
sendmessage(tutu)

print('len(perc_levels)',len(perc_levels))
print('perc_levels_1',perc_levels)
print('总共几格子',step_num)

while True:
  
    time.sleep(2)
    signal=False   
    holding_reduce_mode='off'
    holding_add_mode='off'
    change_iceberg_mode='off'
    modify_mode='off'
    holding_change_mode='off'
    grid_levels_change_mode='off'
    stop_mode='off'
    
    Nowtime=datetime.datetime.now()
    #找出24小时前最高最低价中间价
    
    param_map = { 'instrument_id':swap_instrument_id }
    ticker_result = market_client.get_tickers(param_map)   
    high_24h_new=precfloat(float(ticker_result['data']['high24h']),Tick_digit)
    low_24h_new=precfloat(float(ticker_result['data']['low24h']),Tick_digit)
    total_grids_length_new=precfloat((high_24h_new-low_24h_new),Tick_digit)         #总距离
    middle_price_24h_new=precfloat((high_24h+low_24h)/2,Tick_digit)
    
    
    
    if  high_24h_new != high_24h or low_24h_new!=low_24h :
        #print('high_24h',high_24h)
        change_iceberg_mode='on'
        grid_levels_change_mode='on'
        high_24h=high_24h_new        #全部更新
        low_24h=low_24h_new
        total_grids_length=total_grids_length_new
        middle_price_24h=middle_price_24h_new 
        
        #print('high_24h_new',high_24h_new)
        #print('high_24h_2',high_24h)
    #每五分钟更新一次5-Kline  
    if Nowtime>Fundclock+Fundclockdelta:
        grid_levels_change_mode='on'
    
    for i in range(len(perc_levels)):
        if swap_present_price > perc_levels[0]:                      #穿仓最高值
            grid_levels_change_mode='on'
          
        elif swap_present_price < perc_levels[len(perc_levels)-1]:    #穿仓最低值
            grid_levels_change_mode='on'
      
     
    if grid_levels_change_mode=='on':
        modify_mode='on'
                
        timefor1De = int(round(time.time()*1000))
        dt=str(datetime.datetime.now()-datetime.timedelta(hours=24))
        timefor1Ds= int(time.mktime(time.strptime(dt, '%Y-%m-%d %H:%M:%S.%f')))*1000
        param_map = { 'instrument_id':swap_instrument_id,'start_time':timefor1Ds , 'end_time':timefor1De, 'timeframe_min':'5' }
        kline_result = market_client.get_klines(param_map)
        perc_levels_new=[]
        close_price=[]
        for i in kline_result['data']['close']:
            close_price.append(precfloat(i,Tick_digit))
        grid_width=(np.std(close_price,ddof=1))           #格宽  
        step_num=round(total_grids_length/grid_width)    #暂无用到
        perc_levels_new=[precfloat(x,Tick_digit) for x in np.arange(high_24h, low_24h, -1*grid_width)]     #总距离分几格
        if (high_24h-grid_width*step_num)/grid_width>=0.7:    #判断最后一格格宽是否太小,可否放入
            perc_levels.append(low_24h)
        Fundclock=datetime.datetime.now()             #重新设置一次fundclock  
        perc_levels=perc_levels_new        
        
        #钉钉发送
        tutu= "bit.com网格现在为"+str(perc_levels)
        #sendmessage(tutu)
        #print('现在位置为',location_index)
        #print('perc_levels',perc_levels)
        
  
    #现在价格
    param_map = { 'instrument_id': swap_instrument_id, 'level': '5'}
    swap_depth_result = market_client.get_orderbooks(param_map)
    ask1=precfloat(float(swap_depth_result['data']['asks'][0][0]),Tick_digit)
    bid1=precfloat(float(swap_depth_result['data']['bids'][0][0]),Tick_digit)
    swap_present_price=precfloat((ask1+bid1)/2,Tick_digit)


    #找出在那条线
    for i in range(len(perc_levels)):
        if swap_present_price > perc_levels[0]:                      #穿仓最高值
            stop_mode='on'
            time.sleep(30)    
        elif swap_present_price < perc_levels[len(perc_levels)-1]:    #穿仓最低值
            stop_mode='on'  
            time.sleep(30)    
        elif swap_present_price > perc_levels[i] and i!=0:                   # 如果大于某个网格价
            location_index=i
            break  
    
    if stop_mode=='off':
        #找出在那个中间价的下条线  
        middle_price_index=None   #先乱写
        for j in range(len(perc_levels)):
            if middle_price_24h > perc_levels[j]:                   # 如果大于某个网格价
                middle_price_index = j
                break 

        #计算Target_Amount
        Target_Amount=Total_Money
        ''''''
        #计算本层（过下线)Target_holding_percent
        #Target_holding_percent_downline=location_index/(len(perc_levels)-1)    #先找出当下的location_index 然后算target_holding_percent
        #计算上层（过上线)Target_holding_percent
        #Target_holding_percent_upline=(location_index-1)/(len(perc_levels)-1) 
        ''''''
        #冰山百分比 bit.com没有冰山
       # iceberg_percent=(middle_price_index-1)/(len(perc_levels)-1)
        #找出目前持仓多少
        swap_position = get_position(use_currency)
        position_holding_rate=swap_position/Target_Amount

        if location_index>0:
            if location_index!=len(perc_levels)-1:
                upper_price = perc_levels[location_index-1]
            elif location_index==len(perc_levels)-1:
                upper_price = perc_levels[location_index-2]    #在最后一格的话 卖出线往上挂两格            
            
        if location_index<len(perc_levels):
            lower_price = perc_levels[location_index]    #<<<<<<<<<<<<<<<


       # if upper_price == None:
       #     break
       # if lower_price == None:
        #    break


        #swap_present_price开始移动,移动后才有动作
        if upper_price!= None and swap_present_price > upper_price:
            location_index = location_index-1
            modify_mode='on'    

            #print('location_index',location_index )
            #发钉钉
            tutu= "bit.com网格现在位置在第"+str(location_index)+"格"
            #sendmessage(tutu)

        elif lower_price!= None and swap_present_price < lower_price:
            location_index=location_index+1
            modify_mode='on' 
            #print('location_index',location_index )
            #发钉钉
            tutu= "bit.com网格现在位置在第"+str(location_index)+"格"
            #sendmessage(tutu)
        
        #print('location_index',location_index)
        #print('perc_levels',perc_levels)
        #print('holding_reduce_mode',holding_reduce_mode)
        #print('holding_add_mode',holding_add_mode)
        #print('modify_mode',modify_mode)
        #print('holding_change_mode',holding_change_mode)
        #print('grid_levels_change_mode',grid_levels_change_mode)
        #print('position_holding_rate',position_holding_rate)

        if modify_mode=='on':
            #print('modify_mode',modify_mode)
            #发钉钉
            tutu= "bit.com网格Target_Amount为"+str(Target_Amount)
            #sendmessage(tutu)

            tutu= "bit.com网格现在持仓％为"+str(position_holding_rate)
            #sendmessage(tutu)


            #再找一次上下价格线
            if location_index>0:
                if location_index!=len(perc_levels)-1:
                    upper_price = perc_levels[location_index-1]
                elif location_index==len(perc_levels)-1:
                    upper_price = perc_levels[location_index-2]    #在最后一格的话 卖出线往上挂两格            

            if location_index<len(perc_levels):
                lower_price = perc_levels[location_index]    #<<<<<<<<<<<<<<<

            #再计算一次上下的target_holding_percent
            #计算本层（过下线)Target_holding_percent
            Target_holding_percent_downline=location_index/(len(perc_levels)-1)    #先找出当下的location_index 然后算target_holding_percent
            #计算上层（过上线)Target_holding_percent
            Target_holding_percent_upline=(location_index-1)/(len(perc_levels)-1) 

            tutu= "bit.com网格在该格的持仓target％为"+str(Target_holding_percent_downline)
            #sendmessage(tutu)

            #找出上线的出手size
            #print('position_holding_rate-Target_holding_percent_upline',position_holding_rate-Target_holding_percent_upline)
            if precfloat(position_holding_rate-Target_holding_percent_upline,5)>0:
                holding_change_mode='on'
                upper_reduce_size=round((position_holding_rate-Target_holding_percent_upline)*Target_Amount)
             #   print('upper_reduce_size',upper_reduce_size)
                if upper_reduce_size !=0:
                    holding_reduce_mode='on'
                mode_holding_reduce={'swap_instrument_id':swap_instrument_id,     
                                      'upper_reduce_size':str(upper_reduce_size),        
                                      'upper_price':str(upper_price)}

              #  print('upper_price',upper_price)
                tutu= "bit.com网格upper_price"+str(upper_price)
               # sendmessage(tutu)

            #找出下线的出手size   
           # print('Target_holding_percent_downline-position_holding_rate',Target_holding_percent_downline-position_holding_rate)
            if precfloat(Target_holding_percent_downline-position_holding_rate,5)>0:
                holding_change_mode='on'
                lower_add_size=round((Target_holding_percent_downline-position_holding_rate)*Target_Amount)   
            #    print('lower_add_size',lower_add_size)            
                if lower_add_size !=0 and location_index>=middle_price_index:                    
                    
                    holding_add_mode='on'
                mode_holding_add={'swap_instrument_id':swap_instrument_id,     
                                  'lower_add_size':str(lower_add_size),        
                                  'lower_price':str(lower_price)}

             #   print('lower_price',lower_price)
                tutu= "bit.com网格lower_price"+str(lower_price)
              #  sendmessage(tutu)

            if holding_change_mode=='on':
                #要先撤单,这里是全撤         
                
                param_map = { 'currency':use_currency, 'category':'future'}
                result_uncomplete_order = order_client.get_open_orders(param_map) 
                order_cancel_list=[]
                if len(result_uncomplete_order ['data']) !=0:
                    A=result_uncomplete_order ['data']
                    for i in A:
                        order_cancel_list.append(i)
                    idd=[]
                    for i in order_cancel_list:
                        idd.append(i['order_id']) 
                    if idd != []:
                        for q in range(len(idd)):
                            param_map = { 'currency':use_currency,'category':'future','order_id': idd[q]}        
                            cancel_orders_response=order_client.cancel_orders(param_map)  

                     #下单
                if holding_reduce_mode=='on':
                    holding_reduce_order=take_reduce_order(mode_holding_reduce)    #此时下了单,但不是成交了 .
                   # print('holding_reduce_order_2',holding_reduce_order)
                if holding_add_mode=='on':
                    holding_add_order=take_add_order(mode_holding_add)    #此时下了单,但不是成交了 
                    #print('holding_add_order_2',holding_add_order)


    

perc_levels [59583.5, 58905.82, 58228.14, 57550.46, 56872.77, 56344.0]
location_index 4
Target_Amount_1 3000
position_holding_rate-Target_holding_percent_upline_1 -0.6
Target_holding_percent_downline-position_holding_rate_1 0.8
lower_add_size 2400
holding_add_mode_1 on
holding_add_order {'code': 0, 'message': '', 'data': {'order_id': '274443239', 'created_at': 1620580456525, 'updated_at': 1620580456525, 'user_id': '245396', 'instrument_id': 'BTC-PERPETUAL', 'order_type': 'limit', 'side': 'buy', 'price': '56872.50000000', 'qty': '2400.00000000', 'time_in_force': 'gtc', 'avg_price': '0.00000000', 'filled_qty': '0.00000000', 'status': 'open', 'is_liquidation': False, 'auto_price': '0.00000000', 'auto_price_type': '', 'taker_fee_rate': '0.00050000', 'maker_fee_rate': '0.00010000', 'label': '', 'stop_price': '0.00000000', 'reduce_only': False, 'post_only': False, 'reject_post_only': False, 'mmp': False, 'source': 'api', 'hidden': False}}
len(perc_levels) 6
perc_levels_1 [59583.5, 58905.82, 

In [ ]:
high_24h=precfloat(float(ticker_result['data']['open24h']),Tick_digit)
low_24h=precfloat(float(ticker_result['data']['low24h']),Tick_digit)

In [13]:
location_index

6

In [5]:
lower_add_size=((Target_holding_percent_downline-position_holding_rate)*Target_Amount)

In [6]:
lower_add_size

727.2727272727273

In [7]:
(Target_holding_percent_downline-position_holding_rate)*Target_Amount

727.2727272727273

In [8]:
Target_Amount

2000